# SEC EDGAR Integration - Exploration Notebook

Testing SEC EDGAR API for M&A, insider trading, and beneficial ownership detection.

## 1. Import Libraries

In [ ]:
import requests
import pandas as pd
import json
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta
import time
from typing import Dict, List, Optional
import os

SEC_HEADERS = {
    'User-Agent': 'n8n-sentiment-dashboard/1.0 (contact: admin@example.com)'
}

print('✅ Libraries imported')
print(f'Requests version: {requests.__version__}')

## 2. SEC API Connectivity Test

In [ ]:
print('🔗 Testing SEC API connectivity...\n')

url = 'https://data.sec.gov/api/xbrl/companyfacts/CIK0001067983.json'
print(f'Endpoint: {url}')

try:
    response = requests.get(url, headers=SEC_HEADERS, timeout=10)
    print(f'✅ Status Code: {response.status_code}')
    print(f'✅ Response Size: {len(response.content)} bytes')
    print(f'✅ Content Type: {response.headers.get("content-type")}')
    
    data = response.json()
    print(f'\n✅ JSON Keys: {list(data.keys())}')
    
except Exception as e:
    print(f'❌ Error: {type(e).__name__}: {str(e)}')

## 3. CIK Lookup

In [ ]:
print('🔍 Loading company tickers...\n')

url = 'https://www.sec.gov/files/company_tickers.json'
response = requests.get(url, headers=SEC_HEADERS, timeout=10)
response.raise_for_status()
company_tickers_data = response.json()

ticker_to_cik = {}
for key, company in company_tickers_data.items():
    ticker = company.get('ticker', '').upper()
    cik = str(company.get('cik_str')).zfill(10)
    name = company.get('title')
    ticker_to_cik[ticker] = {'cik': cik, 'name': name}

tickers = ['NVDA', 'PLTR', 'MSFT', 'AAPL']
cik_map = {}

for ticker in tickers:
    if ticker in ticker_to_cik:
        cik_map[ticker] = ticker_to_cik[ticker]
        print(f'✅ {ticker:6} → CIK: {cik_map[ticker]["cik"]} ({cik_map[ticker]["name"]})')
    else:
        print(f'❌ {ticker:6} → Not found')

print(f'\n✅ Results: {len(cik_map)} / {len(tickers)} found')

## 4. Import Production Module

In [ ]:
import sys
from pathlib import Path

notebook_dir = Path.cwd()
project_root = notebook_dir.parent
sys.path.insert(0, str(project_root))

from services.sec_edgar import SecEdgarClient

print(f'✅ Imported SecEdgarClient')
print(f'✅ Module: {SecEdgarClient.__module__}')

prod_client = SecEdgarClient(rate_limit=1.0)
print(f'✅ Created client: {prod_client}')

## 5. Test Production Module

In [ ]:
print('🔍 Testing CIK Lookup\n')

test_tickers = ['NVDA', 'PLTR', 'MSFT', 'AAPL']

for ticker in test_tickers:
    try:
        cik, name = prod_client.lookup_cik(ticker)
        print(f'✅ {ticker:6} → CIK: {cik} ({name})')
    except Exception as e:
        print(f'❌ {ticker:6} → {e}')

print(f'\n✅ Cache size: {len(prod_client.ticker_to_cik_cache)}')

## 6. Form 4 Fetcher

In [ ]:
print('📋 Testing Form 4 Fetcher\n')

df_prod_form4 = prod_client.get_form4_filings('NVDA', days=30, enrich=True)

print(f'✅ Retrieved {len(df_prod_form4)} Form 4 filings for NVDA\n')

if not df_prod_form4.empty:
    print('Enriched Fields:')
    cols = ['ticker', 'filing_date', 'conviction_score', 'actionable']
    print(df_prod_form4[cols].head(5).to_string())
    print(f'\n✅ Form 4 fetcher working')

## 7. 13D & 13G Filings

In [ ]:
print('='*60)
print('Testing 13D & 13G Filings')
print('='*60)

print('\n[13D] Finding M&A activity for NVDA...')
nvda_13d = prod_client.get_13d_filings('NVDA', days=90, enrich=True)
print(f'Found {len(nvda_13d)} 13D filings')

print('\n[13G] Finding ownership changes for NVDA...')
nvda_13g = prod_client.get_13g_filings('NVDA', days=90, enrich=True)
print(f'Found {len(nvda_13g)} 13G filings')

## 8. Summary

In [ ]:
print('\n🎯 SUMMARY')
print('='*60)

summary = {
    '✅ API': 'SEC EDGAR responsive',
    '✅ CIK Lookup': 'Working',
    '✅ Form 4': 'Tested',
    '✅ 13D/13G': 'Queryable',
    '✅ Module': 'Imported',
    '✅ Rate Limit': '1 req/sec',
}

for k, v in summary.items():
    print(f'{k:20} : {v}')

print('\n' + '='*60)